In [0]:
# Imports
import numpy as np
import cv2
import math
from glob import glob
from os import path

In [0]:
class Model:
  def __init__(self, shape, activations):
    self.shape = shape
    self.activations = [np.vectorize(a) for a in activations]
    self.weights = [np.array([np.random.uniform(-1, 1, shape[i+1]) for n in range(shape[i])]) for i in range(len(shape)) if i < len(shape)-1]
    self.biases = [np.random.uniform(-1, 1, shape[i]) for i in range(len(shape))]  
  
  def predict(self, X):
    predictions = []
    for x in X:
      for i in range(len(self.shape)):
        x = np.add(x,self.biases[i])
        x = self.activations[i](x)
        if i < len(self.shape)-1:
          x = np.dot(x, self.weights[i])
      predictions.append(x)
    return predictions
  
  def fit(self, X, Y, epochs, lr):
    for e in range(epochs):
      # Getting the gradients
      Y_hat = self.predict(X)
      #Getting the errors
      errors = np.subtract(Y,Y_hat) * self.activations[-1](Y_hat, prime=True)
      print(errors)
      # Changing the features with respect to their gradients
      """for i in range(len(self.shape)):
        self.biases[i] = np.add((-1.0) * lr * b_grads[i],self.biases[i])
        if i < len(self.shape)-1:
          self.weights[i] = np.add((-1.0) * lr * w_grads[i], self.weights[i])
      """
 

# Get the data


In [0]:
basedir = "/Users/Frederik/dataset"
im_width = 200
im_height = 200 
im_channel = 3

# Get filenames
broadleaf = glob(path.join(basedir,"broadleaf",'*.tif'))
grass = glob(path.join(basedir,"grass",'*.tif'))
soil = glob(path.join(basedir,"soil",'*.tif'))
soybean = glob(path.join(basedir,"soybean",'*.tif'))

# Get images
broadleafX = [cv2.resize(cv2.imread(file),(im_width,im_height)) for file in broadleaf]
grassX = [cv2.resize(cv2.imread(file),(im_width,im_height)) for file in grass]
soilX = [cv2.resize(cv2.imread(file),(im_width,im_height)) for file in soil]
soybeanX = [cv2.resize(cv2.imread(file),(im_width,im_height)) for file in soybean]           
dataX = broadleafX + grassX + soilX + soybeanX

# Get labels
broadleafY = len(broadleaf)*[[1,0,0,0]]
grassY = len(grass)*[[0,1,0,0]]
soilY = len(soil)*[[0,0,1,0]]
soybeanY = len(soybean)*[[0,0,0,1]]                 
dataY = broadleafY + grassY + soilY + soybeanY

# Defining the model

In [36]:
# Defining some activation functions
def sigmoid(x, prime=False):
  if not prime:
    if x < 0:
      return 1/(1+math.exp(x))
    else:
      return 1/(1+math.exp(-x))
  else:
    return sigmoid(x)*(1-sigmoid(x))
  
def ReLu(x, prime=False):
  if not prime:
    return max(0,x)
  else:
    return max(max(0, x),1)

np.random.seed(1)
model = Model(shape=[im_width*im_height,4],activations=[ReLu,sigmoid])

[array([0.99999699, 0.99177483, 1.        , 0.82320016]), array([0.99981504, 1.        , 1.        , 1.        ])]


# Preprocess the data

In [0]:
# Converting images to grayscale
# Converting images to 1 dimensional vectors that are able to be passed as input
dataX = [cv2.cvtColor(im,cv2.COLOR_BGR2GRAY).reshape(-1) for im in dataX]

# Predicting on the data

In [38]:
# Looking at predictions of the data
print(model.predict(dataX[-10:]))

[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
